In [75]:
from extract.extract import getTLEsFromLaunches, getCredentials
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# import sklearn
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import export_graphviz
from sklearn.utils import shuffle

import xgboost as xgb

start = datetime(2016, 1, 1)
end = datetime(2023, 1, 1)

token = getCredentials(source="discos")
username, password = getCredentials(source="spacetrack")

launchIDs = ["2013-066", "2018-092", "2019-084", "2022-002"]
launchIDs = ["2013-066", "2018-092", "2019-084", "2022-002", "2021-006"]
# launchIDs = ["2013-066"]
# launchIDs = ["2022-002"]
# launchIDs = ["2019-084"]
ignoreList = [44827, 44828, 44829, 44830, 44831, 44832] + [39435, 39436, 39437] + [43704, 44046]
# combining everything into one DF
discosDataDict, dfOriginal = getTLEsFromLaunches(
    username,
    password,
    token,
    launchIDs,
    start,
    end,
    combineDiscosAndTLE=True,
    collectLaunches=True,
    collectAllTLEs=True,
    forceRegen=False,
    ignoreList=ignoreList
)

print("(" + ", ".join(launchIDs) + ")")
print(f"total sats: {len(dfOriginal.name.unique())}")


Launch: 2013-066: 100%|██████████| 28/28 [00:02<00:00, 10.95it/s]


Skipped NORADid 39435
Skipped NORADid 39436
Skipped NORADid 39437


Launch: 2018-092: 100%|██████████| 4/4 [00:00<00:00, 19.61it/s]


Skipped NORADid 43704
Skipped NORADid 44046


Launch: 2019-084: 100%|██████████| 1/1 [00:00<00:00, 26.30it/s]


Skipped NORADid 44827
Skipped NORADid 44828
Skipped NORADid 44829
Skipped NORADid 44830
Skipped NORADid 44831
Skipped NORADid 44832


Launch: 2021-006: 100%|██████████| 143/143 [00:03<00:00, 40.60it/s]


(2013-066, 2018-092, 2019-084, 2022-002, 2021-006)
total sats: 281


In [64]:
# print(dfOriginal.columns)
print(f"original length: {len(dfOriginal)}")
df = dfOriginal.query("overallCovariance < 5")#.query("RCS_SIZE == 'SMALL'")
print(f"reduced length: {len(df)}")

colsToDrop = [
       'vimpelId', 'OBJECT_NAME', 'CENTER_NAME',
       'MEAN_MOTION', 'RA_OF_ASC_NODE',
       'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'NORAD_CAT_ID', 'ELEMENT_SET_NO',
       'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT',
        'PERIOD', 'COUNTRY_CODE', 'LAUNCH_DATE', 'DECAY_DATE', 'TLE_LINE0', "OBJECT_TYPE", "objectClass", 
       'TLE_LINE1', 'TLE_LINE2', 'TLE_LINE1min1', 'TLE_LINE2min1']
colsToDrop2 = ["name", "cosparId", "shape", "diameter", "EPOCH", "satno"]
colsToDrop3 = ["ECCENTRICITY", "INCLINATION", "PERIAPSIS", "APOAPSIS", "SEMIMAJOR_AXIS", "deltat"]
colsToDrop4 = ['height', "width", "depth", 'xSectAvg']     
labels = ["errorX", "errorY", "errorZ", "errorCross", "errorAlong", "errorRadial", "overallCovariance", "overallVelocityCovariance", "actualX", "actualY", "actualZ"]

df = df.drop(colsToDrop, axis=1).drop(colsToDrop3, axis=1)
discos = list(discosDataDict.values())[0].drop(['vimpelId', 'cosparId', 'objectClass', 'antennas', 'pannels', 'rods', 'volume'], axis=1)
# print(list(discosDataDict.values())[0].columns)
sampledata = discos.iloc[10]
unit = ["[$m$]", "[-]", "[$m^2$]", "[-]", "[$m$]", "[$m^2$]", "[$m$]", "[$m$]", "[$m^2$]", "[$kg$]", "[$m$]", "[-]"]
description = ["Depth", "Satellite Name", "Average Crosssection", "Shape Description", "Width", "Max Crosssectional Area", "Height", "Diameter", "Minimum Crosssectional Area", "Mass", "Maximum Span Width", "NORAD ID"]
table = pd.DataFrame({"Name in Dataset":discos.columns, "Unit":unit, "Description":description, "Sample Data":sampledata})
print(table)
ltx = table.style.format(precision=3).hide(axis="index")
ltx = ltx.to_latex(buf="latex/a.tex", position="H", hrules=True, clines="skip-last;data", label="discosdata", position_float="centering", caption = "ADD")
print(ltx)
# print(df.groupby("shape").size())

# print(df.satname.unique())

original length: 473898
reduced length: 466872
         Name in Dataset     Unit                  Description  Sample Data
depth              depth    [$m$]                        Depth          0.1
name                name      [-]               Satellite Name     CINEMA 3
xSectAvg        xSectAvg  [$m^2$]         Average Crosssection        0.035
shape              shape      [-]            Shape Description  Box + 1 Rod
width              width    [$m$]                        Width          0.1
xSectMax        xSectMax  [$m^2$]      Max Crosssectional Area     0.043589
height            height    [$m$]                       Height          0.3
diameter        diameter    [$m$]                     Diameter          NaN
xSectMin        xSectMin  [$m^2$]  Minimum Crosssectional Area         0.01
mass                mass   [$kg$]                         Mass          4.0
span                span    [$m$]           Maximum Span Width          0.5
satno              satno      [-]        

In [65]:
discos = list(discosDataDict.values())[0].loc[:, ['antennas', 'pannels', 'rods', 'volume']]
# print(list(discosDataDict.values())[0].columns)
sampledata = discos.iloc[10]
unit = ["[-]", "[-]", "[-]", "[$m^3$]"]
description = ["Numer of Antennas", "Numer of Pannels", "Numer of Rods", "Box Volume"]
table = pd.DataFrame({"Name in Dataset":discos.columns, "Unit":unit, "Description":description, "Sample Data":sampledata})
print(table)
ltx = table.style.format(precision=3).hide(axis="index")
ltx = ltx.to_latex(buf="latex/b.tex", position="H", hrules=True, clines="skip-last;data", label="discosExtras", position_float="centering", caption = "ADD")
print(ltx)

         Name in Dataset     Unit        Description  Sample Data
antennas        antennas      [-]  Numer of Antennas        0.000
pannels          pannels      [-]   Numer of Pannels        0.000
rods                rods      [-]      Numer of Rods        1.000
volume            volume  [$m^3$]         Box Volume        0.003
None


In [66]:
from extract.measure import generateErrorsLabels
tabl = dfOriginal.loc[:, generateErrorsLabels]
sampledata = tabl.iloc[10]
# print(tabl)
# unit = ["[$m$]", "[$m$]", "[$m$]", "[$m$]", "[$m$]", "[$m$]"]
unit = 14 * ["[$km$]"]
description = ["Carthesian error between SGP4 and next TLE estimate"] + 2 * [""] +  ["Cross Track error between SGP4 and next TLE estimate"] + 2 * [""] + ["Position Covariance", "Velocity Covariance","SGP4 position at next TLE Estimate",] + 2 * [""] + ["Actual position at next TLE Estimate"] + 2 * [""]
table = pd.DataFrame({"Name in Dataset":tabl.columns, "Unit":unit, "Description":description, "Sample Data":sampledata})
print(table)
ltx = table.style.format(precision=3).hide(axis="index")
ltx = ltx.to_latex(buf="latex/c.tex", position="H", hrules=True, clines="skip-last;data", label="errorData", position_float="centering", caption = "ADD")
print(ltx)
# print(df.groupby("shape").size())


                                     Name in Dataset    Unit  \
errorX                                        errorX  [$km$]   
errorY                                        errorY  [$km$]   
errorZ                                        errorZ  [$km$]   
errorCross                                errorCross  [$km$]   
errorAlong                                errorAlong  [$km$]   
errorRadial                              errorRadial  [$km$]   
overallCovariance                  overallCovariance  [$km$]   
overallVelocityCovariance  overallVelocityCovariance  [$km$]   
simX                                            simX  [$km$]   
simY                                            simY  [$km$]   
simZ                                            simZ  [$km$]   
actualX                                      actualX  [$km$]   
actualY                                      actualY  [$km$]   
actualZ                                      actualZ  [$km$]   

                                       

In [74]:
table = pd.DataFrame(dfOriginal.groupby("name").first().groupby("shape").count().simX)
table = pd.DataFrame({"Shape":table.index, "Prevalence (\#sats)":table.simX, "Prevalence (\%)":table.simX / len(dfOriginal.name.unique()) * 100 })
# print(table)
# print(table)
ltx = table.style.format(precision=1).hide(axis="index")
ltx = ltx.to_latex(buf="latex/d.tex", position="H", hrules=True, clines="skip-last;data", label="tab:shapeTypes", position_float="centering", caption = "ADD")
print(ltx)

None
